# 数据探索与清洗

因为按照常理助学金是给家庭有经济困难的学生的，而学生的消费情况比较能反映学生的经济状况，所以本次探索会先清洗出学生的各类消费情况及总消费。
学习成绩也应该会对奖学金的评定有一定的影响，以我在本科读书时的经验来看，成绩好的能拿奖学金一般不会申请或者不能通过助学金的审批，而曾经太差也会影响，所以我第二步准备将学生成绩清洗为在学院的排名，及学院人数这两个变量。

### 步骤
#### step1 清晰学生饭卡数据

#### step2 清洗学生学习成绩数据

## 处理学生消费数据

In [1]:
import pandas as pd 
import warnings 
warnings.filterwarnings("ignore")
names_card = [ '学生id','消费类别','消费地点','消费方式','消费时间','消费金额','剩余金额']
# 导入数据
card_train = pd.read_csv("../input/card_train.txt",header=None,encoding='utf-8',names = names_card)
card_test = pd.read_csv("../input/card_test.txt",header=None,encoding='utf-8',names = names_card)
card_data = pd.concat([card_train,card_test])

del card_test,card_train

In [6]:
card_data['消费方式'] = card_data['消费方式'].astype('category')
card_data['消费方式'].describe()

count     23998315
unique          11
top             食堂
freq       9027893
Name: 消费方式, dtype: object

In [8]:
print(card_data['消费方式'].unique())

[淋浴, 开水, 其他, 洗衣房, 文印中心, ..., 食堂, 校车, NaN, 超市, 校医院]
Length: 12
Categories (11, object): [淋浴, 开水, 其他, 洗衣房, ..., 食堂, 校车, 超市, 校医院]


数据总共记录了11类消费数据（不含Nan），23998315次消费，其中食堂消费次数最多

In [9]:
# 缺失值处理
# 查看Ｎａｎ
card_data.isnull().sum()

学生id         0
消费类别     62723
消费地点    850087
消费方式    850087
消费时间         0
消费金额         0
剩余金额         0
dtype: int64

In [18]:
# 由于消费方式中食堂消费最多，使用食堂代替缺失的消费类别
card_data['消费方式'].fillna('食堂',inplace=True)

In [11]:
#每个学生的总消费
card_sum_by_ID = card_data.groupby('学生id')['消费金额'].sum()

In [12]:
card_sum_by_ID.head(20)

学生id
0      4997.56
1      6182.69
2      4966.65
3        42.40
8     11472.37
9     14502.91
10     6028.74
11     5976.20
16     2974.56
17      495.85
19     3331.06
20     6393.73
21     3165.76
22    11328.89
23    10959.76
28     6401.58
29     3845.78
32     2093.69
33       97.52
34     7647.02
Name: 消费金额, dtype: float64

In [22]:
## 每个学生各类别的消费
card_sum_by_ID_type = card_data.groupby(['学生id','消费方式'])['消费金额'].sum().unstack('消费方式')

In [23]:
card_sum_by_ID_type.head()

消费方式,其他,图书馆,开水,教务处,文印中心,校医院,校车,洗衣房,淋浴,超市,食堂
学生id,,,,,,,,,,,
0,NaN,84.4,429.58,NaN,0.3,NaN,417.49,NaN,12.30,839.68,3213.81
1,NaN,324.2,265.97,29.3,96.4,NaN,119.40,25.02,22.50,514.30,4785.60
2,0.01,22.4,670.36,NaN,NaN,NaN,34.88,22.20,53.10,829.95,3333.75
3,NaN,NaN,0.18,NaN,NaN,NaN,NaN,7.62,34.60,NaN,NaN
8,NaN,425.9,2440.94,NaN,2.6,NaN,2211.45,4.50,39.83,10.00,6337.15


In [25]:
# 缺失值
card_sum_by_ID_type.fillna(0,inplace=True)

In [26]:
card_sum_by_ID_type.head()

消费方式,其他,图书馆,开水,教务处,文印中心,校医院,校车,洗衣房,淋浴,超市,食堂
学生id,,,,,,,,,,,
0,0.00,84.4,429.58,0.0,0.3,0.0,417.49,0.00,12.30,839.68,3213.81
1,0.00,324.2,265.97,29.3,96.4,0.0,119.40,25.02,22.50,514.30,4785.60
2,0.01,22.4,670.36,0.0,0.0,0.0,34.88,22.20,53.10,829.95,3333.75
3,0.00,0.0,0.18,0.0,0.0,0.0,0.00,7.62,34.60,0.00,0.00
8,0.00,425.9,2440.94,0.0,2.6,0.0,2211.45,4.50,39.83,10.00,6337.15


In [32]:
## 结合数据
card = pd.concat([card_sum_by_ID,card_sum_by_ID_type],axis =1)

In [36]:
del card_sum_by_ID,card_sum_by_ID_type,card_data


## 处理学生学习成绩数据

In [37]:
name_score = ['学生id','学院编号','成绩排名']
# 导入数据
train = pd.read_csv("../input/score_train.txt",header=None,encoding='utf-8',names = name_score)
test = pd.read_csv("../input/score_test.txt",header=None,encoding='utf-8',names = name_score)
s_data = pd.concat([train,test])

In [44]:
s_data.groupby('学院编号').成绩排名.max()

学院编号
1      370
2      398
3     2304
4     2416
5     2775
6     1570
7      963
8     2830
9     2933
10     747
11     613
12     513
13    2714
14     582
15     631
16     406
17    2051
18     193
19    2305
Name: 成绩排名, dtype: int64

In [50]:
s_data.isnull().sum()


学生id    0
学院编号    0
成绩排名    0
dtype: int64

In [51]:
print(s_data.notnull().sum())
print(card.其他.count())

学生id    18130
学院编号    18130
成绩排名    18130
dtype: int64
21631


In [53]:
test.notnull().sum()

学生id    9000
学院编号    9000
成绩排名    9000
dtype: int64

#### 发现问题：　有消费记录的学数21631而有学习成绩记录的学生数18130，有3501人没有排名数据
处理方案待定

In [54]:
transfored_score = s_data.groupby('学院编号').成绩排名.transform(lambda x: (x - x.mean()) / x.std())

In [55]:
transfored_score 

0      -1.681326
1      -1.680156
2       1.749205
3       1.760045
4      -1.709162
5      -1.707670
6      -1.647174
7      -0.412101
8      -0.323628
9       0.370275
10      0.881042
11      0.991709
12      0.994141
13      1.020895
14      1.024544
15      1.137642
16      1.151019
17      1.220338
18      1.231283
19      1.249524
20      1.262902
21      1.667774
22      1.668944
23      1.677133
24      1.678302
25      1.688830
26      1.692340
27      1.698189
28      1.704038
29      1.716905
          ...   
8970    1.606283
8971    1.611367
8972    1.617722
8973    1.618993
8974    1.621535
8975    1.625347
8976    1.627889
8977    1.629160
8978    1.638057
8979    1.646953
8980    1.648224
8981    1.653308
8982    1.654579
8983    1.660934
8984    1.662205
8985    1.663476
8986    1.664747
8987    1.671101
8988    1.673643
8989    1.674914
8990    1.676185
8991    1.688895
8992    1.691437
8993    1.692707
8994    1.005086
8995    1.007518
8996    0.367005
8997    0.3717

In [ ]:
# http://pandas.pydata.org/pandas-docs/stable/io.html#io-read-csv-table